In [ ]:
import os
print(os.getcwd())
import sys
print(sys.path)


# Verification of JPK_V7K (Quarterly) Implementation

This notebook verifies the current implementation of JPK_V7K (Quarterly) version 3 (Schema 2025/12/19).

**Goal:** Identify if the implementation correctly handles:
1.  Namespace and Schema Version.
2.  Header (`Naglowek`) structure.
3.  Declaration (`Deklaracja`) structure.
4.  Submission logic (Monthly vs Quarterly aggregation).


In [ ]:
import sys
import os
import xml.etree.ElementTree as ET

# Add project root to path
sys.path.append(os.getcwd())

from logic.jpk_service import JpkService
from database.engine import SessionLocal
from database.models import CompanyConfig

# Create a session
session = SessionLocal()

# Instantiate JpkService
jpk_service = JpkService(session)

# Dummy check if CompanyConfig exists
company = session.query(CompanyConfig).first()
if not company:
    print("WARNING: No CompanyConfig found. Creating dummy for test.")
    company = CompanyConfig(
        company_name="Test Company",
        nip="1234567890",
        tax_office_code="1234",
        is_natural_person=False
    )
    session.add(company)
    session.commit()

print(f"Testing with Company: {company.company_name}, NIP: {company.nip}")
print(f"Current JPK Namespace in Service: {jpk_service.NS}")


In [ ]:
# Generate JPK for Q1 2026 (assuming current implementation behavior)
test_file = "jpk_v7k_q1_2026_test.xml"
try:
    jpk_service.generate_jpk_v7m(2026, 1, test_file, quarter=1)
    
    # Parse the generated XML
    tree = ET.parse(test_file)
    root = tree.getroot()
    
    ns = {"jpk": "http://crd.gov.pl/wzor/2025/12/19/14089/"} # Expected JPK_V7K (3) Namespace

    # Check Namespace
    print(f"Root Namespace: {root.tag.split('}')[0].strip('{')}")
    if root.tag.split('}')[0].strip('{') == ns['jpk']:
        print("PASS: Namespace MATCHES expected JPK_V7K (3) namespace.")
    else:
        print(f"FAIL: Namespace mismatch. Expected {ns['jpk']}")

    # Check Header
    header = root.find("jpk:Naglowek", ns)
    if header is not None:
        kwartal = header.find("jpk:Kwartal", ns)
        miesiac = header.find("jpk:Miesiac", ns)
        kod_form = header.find("jpk:KodFormularza", ns)
        
        if kwartal is not None:
            print(f"Header uses <Kwartal>: {kwartal.text} (Correct for quarterly, BUT only if submitted quarterly? Wait. Monthly submissions use Miesiac?)")
        if miesiac is not None:
            print(f"Header uses <Miesiac>: {miesiac.text}")
            
        if kod_form is not None:
            print(f"KodFormularza: {kod_form.text}, kodSystemowy: {kod_form.attrib.get('kodSystemowy')}")

    # Check Declaration
    declaration = root.find("jpk:Deklaracja", ns)
    if declaration is not None:
        print("Declaration PRESENT (As expected for end of quarter or full quarter file).")
        # Check specific fields
    else:
        print("Declaration ABSENT.")

    # Check Ewidencja (Sales Rows)
    # We need to know if it aggregated invoices. 
    # Since we didn't add invoices, it will be empty.
    # But checking code logic (static analysis or modify logic to print date range) confirms it aggregates.
    
except Exception as e:
    print(f"Error generating JPK: {e}")
